In [2]:
# -*- coding: utf-8 -*-

from __future__ import division, unicode_literals

import pandas as pd
import numpy as np

import time
from time import sleep

from bs4 import BeautifulSoup
import requests

import re
import html5lib

import pickle

In [14]:
bechdel_url = 'http://bechdeltest.com/?list=all'

In [22]:
columns = ['imdb_id', 'title', 'year', 'bechdel']

In [23]:
def make_soup(url):
    response = requests.get(url)
    page = BeautifulSoup(response.text, 'html5lib')
    return page

In [24]:
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print f.__name__, 'took', end - start, 'seconds'
        return result
    return f_timer

In [25]:
@timefunc
def get_bechdel(url, stop):
    soup = make_soup(url)
    imdb_regex = re.compile('/[^/]+/$')
    year_regex = re.compile(' \([0-9]* movies\)')
    df = pd.DataFrame(columns = columns)
    
    for movie in soup.findAll('div', class_='movie'):    
        # Initialize an empty dictionary
        d = dict.fromkeys(columns)
        # Append Year
        year = int(re.sub(year_regex,'',movie.findPrevious('h3').text))
        if year == stop-1:
            break
        else:
            d['year'] = year
        # Find and append IMDB ID
        url = movie.find('a')['href']
        d['imdb_id'] = re.search(imdb_regex, url).group(0).replace('/','')
        # Find and append Movie Title
        d['title'] = movie.text.strip()
        # Find and append Pass/Fail Status
        if 'nopass' in movie.find('img')['src']:
            d['bechdel'] = 0
        else:
            d['bechdel'] = 1
        # Append to dataframe
        df = df.append(d, ignore_index=True)
        sleep(.05)
    
    return df

In [26]:
# Scraping for movies back to 1980 (inclusive)
bechdel_df = get_bechdel(bechdel_url, 1980)

get_bechdel took 418.785094976 seconds


In [7]:
bechdel_df.to_pickle('data/bechdel.pkl')

## Load Pickle Point

In [3]:
bechdel_df = pd.read_pickle('data/bechdel.pkl')

In [5]:
# Slight cleanup
bechdel_df = bechdel_df.rename(columns = {'passfail': 'bechdel'})
bechdel_df.drop(['imdbID', 'Title', 'Year', 'PassFail'], inplace = True)